# **Latent Semantic Analysis (Tugas5)**

Topic modelling adalah salah satu teknik analisis teks pada data science yang bertujuan untuk mengidentifikasi pola atau topik-topik yang tersembunyi di dalam kumpulan dokumen atau korpus teks.

LSA atau Latent Semantic Analysis adalah teknik analisis teks pada data science yang menggunakan metode matematika multivariat untuk mengidentifikasi pola hubungan antara kata-kata dalam dokumen atau korpus teks. LSA menganggap bahwa kata-kata yang sering muncul bersama-sama dalam dokumen memiliki hubungan semantik yang kuat, sehingga dapat digunakan untuk mengidentifikasi topik dan hubungan antar topik dalam dokumen atau korpus.

## Crawling Comment Youtube

In [6]:
# !pip install selenium --quiet
# !pip install selenium
# !pip install webdriver_manager

In [7]:
# import time
# from selenium.webdriver import Chrome
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.chrome.service import Service

# # selenium 4
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [8]:
# data=[]

# with Chrome(executable_path=r'C:\Program Files\chromedriver.exe') as driver:
#     wait = WebDriverWait(driver,10)
#     driver.get("https://www.youtube.com/watch?v=rbZ3Wv_hLJA&ab_channel=CNNIndonesia")

#     for item in range(10): 
#         wait.until(EC.visibility_of_element_located((By.TAG_NAME, "body"))).send_keys(Keys.END)
#         time.sleep(5)

#     for comment in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content"))):
#         data.append(comment.text)

In [9]:
# import pandas as pd   
# df = pd.DataFrame(data, columns=['comment'])
# df

### Preprocessing Data

In [10]:
# import re, string
# import numpy as np

# # Text Cleaning
# def cleaning(text):
#     # HTML Tag Removal

#     text = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});').sub('', str(text))

#     # Case folding
#     text = text.lower()

#     # Trim text
#     text = text.strip()

#     # Remove punctuations, karakter spesial, and spasi ganda
#     text = re.compile('<.*?>').sub('', text)
#     text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
#     text = re.sub('\s+', ' ', text)

#     # Number removal
#     text = re.sub(r'\[[0-9]*\]', ' ', text)
#     text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
#     text = re.sub(r'\d', ' ', text)
#     text = re.sub(r'\s+', ' ', text)

#     # Mengubah text 'nan' dengan whitespace agar nantinya dapat dihapus
#     text = re.sub('nan', '', text)

#     return text

In [11]:
# from indoNLP.preprocessing import pipeline, replace_word_elongation, replace_slang, emoji_to_words, remove_html
# pipe = pipeline([replace_word_elongation, replace_slang, emoji_to_words, remove_html])
# # steamming
# from nltk.stem import PorterStemmer
# ps = PorterStemmer()

In [12]:
# df['comment (clean)'] = df['comment'].apply(lambda x: pipe(x))
# df['comment (clean)'] = df['comment (clean)'].apply(lambda x: cleaning(x))
# # ubah empty string menjadi NaN
# df['comment (clean)'] = df['comment (clean)'].replace('', np.nan)

In [13]:
# print(df.isna().sum())

In [14]:
# df.dropna(inplace=True)
# df

In [15]:
# to csv
# df.to_csv('datacomment_cnn_fix.csv', index=False)

## Implementasi ke Modelling

In [16]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Import data CSV

In [17]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/masaul/data-csv/main/comment_yt.csv')
df

,comment,comment (clean)
0,ID\nSkip navigation\nSign in\nWorship Piano: B...,id skip navigation sign ini worship piano begi...
1,"Pak Prabowo Subianto saya mohon,\nAnda jangan ...",pak prabowo subianto saya mohon anda jangan be...
2,gw dan sekeluarga adalah loyalis PDI Perjuanga...,gue dan sekeluarga adalah loyalis pdi perjuang...
3,"ganjar salah satu yang menolak israel, berakib...",ganjar salah satu yang menolak israel berakiba...
4,"Pak Prabowo sekarang auranya adem,beda waktu n...",pak prabowo sekarang auranya adem beda waktu n...
...,...,...
1160,Di,di
1161,Siapa pula yg mau pilih prabowo....history mas...,siapa pula yang mau pilih prabowo history masa...
1162,Lembaga survei bayaran\nAslinya pak Anies yg t...,lembaga survei bayaran aslinya pak anies yang ...
1163,Siapapun calonya presiden 2024\nKami rakyat t...,siapapun calonya presiden kami rakyat tetap an...


In [18]:
import nltk
nltk.download('stopwords', quiet=True)

True

In [19]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['comment (clean)'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

### Bobot Kata Terhadap Masing-Masing Topik

In [20]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

Topic 0:  [('prabowo', 0.7521363053722471), ('ganjar', 0.3371991592275814)]
Topic 1:  [('ganjar', 0.7502868233726872), ('pilih', 0.18697888908418656)]
Topic 2:  [('mahfud', 0.5852118452305931), ('md', 0.5178504678657214)]
Topic 3:  [('anis', 0.569265749836039), ('pilih', 0.38146262585649654)]


### Bobot Setiap Topik Terhadap Dokumen

In [21]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["comment (clean)"], df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

,comment (clean),Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,id skip navigation sign ini worship piano begi...,0.152410,0.051521,-0.010232,0.048767,0
1,pak prabowo subianto saya mohon anda jangan be...,0.307140,0.188706,0.027650,-0.118431,0
2,gue dan sekeluarga adalah loyalis pdi perjuang...,0.054229,0.070597,-0.006094,0.014747,1
3,ganjar salah satu yang menolak israel berakiba...,0.071630,0.146403,0.047237,-0.040582,1
4,pak prabowo sekarang auranya adem beda waktu n...,0.098515,-0.021353,-0.033110,-0.032312,0
...,...,...,...,...,...,...
1160,di,0.000000,0.000000,0.000000,0.000000,0
1161,siapa pula yang mau pilih prabowo history masa...,0.107988,0.008454,-0.050866,0.057600,0
1162,lembaga survei bayaran aslinya pak anies yang ...,0.030143,0.034598,-0.037297,0.112036,3
1163,siapapun calonya presiden kami rakyat tetap an...,0.118884,0.049294,-0.130737,0.280394,3


In [22]:
df_lsa['Topik'].value_counts()

0    559
3    329
1    217
2     60
Name: Topik, dtype: int64